## Model Evaluation (Bootstrapped Samples)
#### Goal:
To evaluate model discrimination and calibration of tuned SVM, random forest, and gradient boosted trees classifiers and logistic regression model

#### Input(s):

CSV file with preprocessed narrative text (processed_narr_{batch_date}.csv, output of run_preprocessing.ipynb)
File with case/control flags ('E:/Data Science Demonstration Project/SUDORSdataTruth.xlsx')
File with tuned hyperparameters for each model (tuned_params_{batch_date}.pickle, output of hyperparameter_tuning.ipynb)
#### Output(s):

95% CIs for each discrimination and calibration metric (saved in file)
SVM, RF, and XGB models achieving median discrimination
Top SHAP values for median random forest and gradient boosted trees classifiers (saved in file)
SHAP plots for top 20 features
#### To run, set 2 variables and make sure correct input files are specified in first cell:

my_directory = where you want outputs to save (e.g., 'C:/Users/dc20b49/Documents/TDH_DS_Demo/')
batch_date = batch_date used in 1_preprocessing_pipeline.ipynb (e.g, '8-4-22')
## Setup

In [ ]:
import pandas as pd
import pickle

my_directory = 'C:/Users/dc20b46/Documents/tndh_ds_demo/'
batch_date = '8-10-22'

print(f'''my_directory = {my_directory},
batch_date = {batch_date}''')

### load data
# tuned hyperparameters
# with open(f'{my_directory}/tuned_params_{batch_date}.pickle', 'rb') as handle:
#     tuned_params = pickle.load(handle)
tuned_params = {
    'RF': {'max_depth': 37, 'min_samples_leaf': 1, 
           'min_samples_split': 4, 'n_estimators': 350, 'oob_score': True},
    'XGB': {'max_depth': 4, 'min_child_weight': 2, 'eta': 0.25,
            'reg_alpha': 0.01, 'reg_lambda': 0.25, 'objective': 'binary:hinge',
            'eval_metric': 'error', 'colsample_bytree': 0.5},
    'SVM': {'kernel': 'rbf', 'C': 10, 'gamma': 0.1}
}

# preprocessed narratives
narr_df = pd.read_csv(f'{my_directory}/processed_narr_{batch_date}.csv')
print(narr_df.shape)

# case-control flags
#cases = pd.read_csv(f'{my_directory}/')
cases = pd.read_excel('E:/Data Science Demonstration Project/SUDORSdataTruth.xlsx')
print(cases.shape)

print('Data loaded')


## Prepare Test and Train Datasets

In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import re

def transform_data(count_matrix, vectorizer, idx):
    count_array = count_matrix.toarray()
    df_transformed = pd.DataFrame(data=count_array, columns = vectorizer.get_feature_names_out())
    df_transformed = df_transformed[[c for c in df_transformed.columns if not re.match(r'^\d+$', c) and len(c) > 3]]
    df_transformed.set_index(idx, inplace = True)
    return df_transformed

# assign case/control flags
narr_df['case'] = narr_df['DID'].isin(cases.DID.unique()).astype(int)

# remove autopsies < 100 characters
print(f'''Autopsies removed: {np.sum(narr_df.full_narr_lemma_text_len < 100)} ''')
narr_df = narr_df.loc[narr_df.full_narr_lemma_text_len >= 100]

# set test/train data
narr_df['train'] = narr_df['year'].apply(lambda x: x < 2021).astype(int)
narr_df.set_index('DID', inplace = True)

# shuffle data
shuffled_df = shuffle(narr_df, random_state = 0)

# DIDs for train, calibration, and test sets
all_train_DID = shuffled_df.loc[shuffled_df.train==1].index
train_DID, calib_DID = train_test_split(all_train_DID, test_size = 0.05, random_state = 0)
test_DID = shuffled_df.loc[shuffled_df.train==0].index

# get labels
ytrain, ycalib, ytest = shuffled_df['case'].loc[train_DID], shuffled_df['case'].loc[calib_DID], shuffled_df['case'].loc[test_DID]

### TF-IDF
# create vocabulary based on training set
tfidf_vect = TfidfVectorizer(min_df=20)
tfidf_matrix = tfidf_vect.fit_transform(shuffled_df.loc[train_DID]['full_narr_lemma_text'].values.astype('U'))
tfidf_train = transform_data(tfidf_matrix, tfidf_vect, train_DID)

# calibration
tfidf_matrix_calib = tfidf_vect.transform(shuffled_df.loc[calib_DID]['full_narr_lemma_text'])
tfidf_calib = transform_data(tfidf_matrix_calib, tfidf_vect, calib_DID)

# test
tfidf_matrix_test = tfidf_vect.transform(shuffled_df.loc[test_DID]['full_narr_lemma_text'])
tfidf_test = transform_data(tfidf_matrix_test, tfidf_vect, test_DID)

# datasets
datasets = {'TFIDF': [tfidf_train, tfidf_calib, tfidf_test]}

print(f'''
Vocabulary: {tfidf_train.shape}
Total: {shuffled_df.shape[0]}
Train: {tfidf_train.shape[0]}
Calibration: {tfidf_calib.shape[0]}
Test: {tfidf_test.shape[0]}
''')

## Train Models on Bootstrapped Samples

In [ ]:
# bootstrap samples
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, accuracy_score, fbeta_score
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

def bs_samples(x, y):
    rand_idx = np.random.choice(x.index, replace = True, size = x.shape[0])
    x_bs, y_bs = x.loc[rand_idx], y.loc[rand_idx]
    return x_bs, y_bs

def get_metrics(method, mod, X_test, y_test):
    preds = mod.predict(X_test)
    AUROC = roc_auc_score(y_test, preds)
    precision = precision_score(y_test, preds)
    recall = recall_score(y_test, preds)
    f1= f1_score(y_test, preds)
    f2 = fbeta_score(y_test, preds, beta=2)
    accuracy = mod.score(X_test, y_test)
    return [method, AUROC, precision, recall, f1, f2, accuracy]

xtrain, xcalib, xtest = datasets['TFIDF']

print(f'''
Train: {xtrain.shape[0]}
Calibration: {xcalib.shape[0]}
Test: {xtest.shape[0]}
''')


In [ ]:
%%time
models = {}
models['Logistic'], models['SVM'], models['RF'], models['XGB'] = {}, {}, {}, {}
for i in range(1):
    print(f'Starting boostrap #{i+1}')
    
    # sample from train data
    xtrain_bs, ytrain_bs = bs_samples(xtrain, ytrain)
    
    ### fit models
    # logistic regression
    print(f'Training logistic regression model...')
    lr = LogisticRegression(max_iter=1000)
    lr.fit(xtrain_bs, ytrain_bs)
    models['Logistic'].update({i: lr})
  
    # svm
    print(f'Training SVM model...')
    svm_mod = svm.SVC(**tuned_params['SVM'])
    svm_mod.fit(xtrain_bs, ytrain_bs)
    models['SVM'].update({i: svm_mod})

    # random forest
    print(f'Training random forest model...')
    rf_mod = RandomForestClassifier(**tuned_params['RF'])
    rf_mod.fit(xtrain_bs, ytrain_bs)
    models['RF'].update({i: rf_mod})
    
    # xgb
    print(f'Training XGBoost model...')
    xgb_mod = xgb.XGBClassifier(**tuned_params['XGB'])
    xgb_mod.fit(xtrain_bs, ytrain_bs)
    models['XGB'].update({i: xgb_mod})

## Evaluate Model Discrimination on Bootstrapped Samples

In [ ]:
%%time

# calculate metrics for each model
results_bs = []
test_bs = {}
for i in range(100):
    
    if i % 10 == 0:
        print(i)
        
    # sample from test data
    xtest_bs, ytest_bs = bs_samples(xtest, ytest)
    test_bs.update({i: [xtest_bs, ytest_bs]})
    
    ### evaluate model discrimination
    for method in models.keys():
        metrics_bs = get_metrics(method = method, mod = models[method][i],
                                 X_test = xtest_bs, y_test = ytest_bs)
        results_bs.append(metrics_bs)

results_bs = pd.DataFrame(results_bs, columns = ['Method', 'AUROC', 'Precision', 
                                                 'Recall', 'F1', 'F2', 'Accuracy'])
results_bs

In [ ]:
# calculate metrics across all boostrapped samples
results_CIs = results_bs.groupby(['Method']).quantile([0.025, 0.5, 0.975])
results_CIs

## Save Median Bootstrapped Models

In [ ]:
from joblib import dump

### make i=model with median discrimination
for key in models.keys():
    xtest_bs, ytest_bs = test_bs[i]
    median_model = {'Model': models[key][i],
                    'xtest': xtest_bs,
                    'ytest': ytest_bs}
    ## save model
    dump(median_model, f'{my_directory}/median_{key}_{batch_date}.joblib') 

## Evaluate Model Calibration on Bootstrapped Samples

In [ ]:
%%time
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import brier_score_loss

# calibrate models and generate calibrated probabilities
calib_y_preds = pd.DataFrame()
brier_score = []
for i in range(100):
    
    if i % 10 == 0:
        print(i)
    
    # load test data
    xtest_bs, ytest_bs = test_bs[i]
    
    # sample from calibration data
    xcalib_bs, ycalib_bs = bs_samples(xcalib, ycalib)

    ### calibrate models
    for method in models.keys():
        mod_calib = CalibratedClassifierCV(models[method][i], cv='prefit')
        mod_calib.fit(xcalib_bs, ycalib_bs)
        
        # generate calibrated predictions
        calib_preds = mod_calib.predict_proba(xtest_bs)[:,1]
        calib_bs = pd.DataFrame({'method': 'RF',
                                 'i': i,
                                 'y': test_bs[0][1].values, 
                                 'pred': calib_preds})
        calib_y_preds = pd.concat([calib_y_preds, calib_bs], ignore_index = True)
        
        # calculate brier score
        brier_score_bs = brier_score_loss(ytest_bs, calib_preds)
        brier_score.append([method, brier_score_bs])

In [ ]:
pd.DataFrame(brier_score, columns = ['Method', 'Brier_Score'])

In [ ]:
### export calibration probabilities to R for Spiegelhalter z-test
calib_y_preds.head()

## Top SHAP Features of Median Bootstrapped Model

In [ ]:
# load median models
from joblib import load, dump
import shap

rf_model = load(f'{my_directory}/median_rf_{batch_date}.joblib')
xgb_model = load(f'{my_directory}/median_xgb_{batch_date}.joblib')

print('Models loaded')

In [ ]:
%%time

# SHAP values for median RF
print('Generating SHAP explainer...')
rf_explainer = shap.Explainer(rf_model['Model'].predict,
                              rf_model['xtest'], max_evals=7000) 

shap_values_rf = rf_explainer(rf_model['xtest'], max_evals=7000)

# save SHAP values
shap_rf_filename = f'{my_directory}/explainer_rf_{batch_date}.bz2'
dump(shap_values_rf, filename = shap_rf_filename, compress=('bz2', 9))

In [ ]:
%%time

# load shap values
try:
    type(shap_values_rf)
except:
    print('Loading SHAP explainer...')
    shap_rf_filename = f'{my_directory}/explainer_rf_{batch_date}.bz2'
    shap_values_rf = load(filename = shap_rf_filename)

print('Generating SHAP plot...')
shap.plots.beeswarm(shap_values_rf, max_display=20, show = False)
plt.savefig(f'{my_directory}/rf_shap_{batch_date}.svg', 
            format='svg', dpi=1200, bbox_inches='tight')

In [ ]:
%%time

# SHAP values for median XGB
print('Generating SHAP explainer...')
xgb_explainer = shap.TreeExplainer(xgb_model['Model'])
shap_values_xgb = xgb_explainer(xgb_model['xtest'])

# plot SHAP values for top 20 features
print('Generating SHAP plot...')
shap.plots.beeswarm(shap_values_xgb, max_display = 20, show = False)
plt.savefig(f'{my_directory}/xgb_shap_{batch_date}.svg', 
            format='svg', dpi=1200, bbox_inches='tight')